In [3]:
# chapter 5-1 データセットの取得と活用

In [4]:
# Wikipediaのデータセットのダウンロード

In [1]:
# !wget https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles1.xml-p1p168815.bz2
# Wikipediaのデータセット(ダンプファイル)の取得

In [2]:
# ! bzcat jawiki-latest-pages-articles1.xml-p1p168815.bz2 | less
# 解凍せず中身を閲覧可能
# j: 下スクロール, k: 上スクロール, q: 終了

In [5]:
# !wget https://github.com/attardi/wikiextractor/raw/master/WikiExtractor.py
# WikiExtractor.py スクリプトをインストール

In [6]:
# !python WikiExtractor.py --no-templates -o articles -b 100M jawiki-latest-pages-articles1.xml-p1p168815.bz2
# スクリプトを用いてダンプファイルのマークアップを取り除きテキストに変換

In [7]:
# !tree articles/
# ディレクトリの確認

In [8]:
# 自然言語処理技術を用いた頻出単語の抽出

In [9]:
# !brew install mecab-ipadic
# MeCabのインストール

In [10]:
# !mecav -v
# バージョンの確認

In [11]:
# !pip install mecab-python3
# Python3対応バインディングのインストール(MeCab公式ではない。公式は未対応?)

In [12]:
# MeCabをPythonから使う

In [15]:
%%writefile mecab_sample.py

# MeCabをPythonから使う

import MeCab

tagger = MeCab.Tagger()
tagger.parse('') # これは.parseToNode()の不具合を回避するのに必要

# .parseToNode()で最初の形態素を表すNodeオブジェクトを取得する
node = tagger.parseToNode('すもももももももものうち')

while node:
    # .surfaceは形態素の文字列、.featureは品詞などを含む文字列をそれぞれ表す
    print(node.surface, node.feature)
    node = node.next # .nextで次のNodeを取得する

Overwriting mecab_sample.py


In [16]:
!python mecab_sample.py

 BOS/EOS,*,*,*,*,*,*,*,*
すもも 名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも 名詞,一般,*,*,*,*,もも,モモ,モモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも 名詞,一般,*,*,*,*,もも,モモ,モモ
の 助詞,連体化,*,*,*,*,の,ノ,ノ
うち 名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
 BOS/EOS,*,*,*,*,*,*,*,*


In [17]:
# 以下でもほぼ同様の結果が得られる
# !echo すもももももももものうち | mecab

In [18]:
# 文章から頻出単語を抽出する

In [19]:
# Wikipediaの文章から頻出単語を抜き出す

In [21]:
%%writefile word_frequency.py

# Wikipediaの文章から頻出単語を抜き出す

import sys
import os
from glob import glob
from collections import Counter
import MeCab

def main():
    """
    コマンドライン引数で指定したディレクトリ内のファイルを読み込んで頻出単語を抽出する
    """
    
    input_dir = sys.argv[1] # コマンドラインの第1引数でWikiExtractorの出力先のディレクトリを指定する
    
    tagger = MeCab.Tagger('')
    tagger.parse('') # .parseToNode()の不具合を回避するのに必要
    # 単語の頻度を格納するCounterオブジェクトを作成する
    # Counterクラスはdictを継承しており、値としてキーの出現回数を保持する
    frequency = Counter()
    count_proccessed = 0
    
    # glob()でワイルドカードにマッチするファイルのリストを取得し、マッチしたすべてのファイルを処理する
    for path in glob(os.path.join(input_dir, '*', 'wiki_*')):
        print('Proccessing {0}...'.format(path), file=sys.stderr)
        
        with open(path) as file: # ファイルを開く
            for content in iter_docs(file): # ファイル内の全記事について反復処理する
                tokens = get_tokens(tagger, content) # ページから名詞のリストを取得する
                # Counterのupdate()メソッドにリストなどの反復可能オブジェクトを指定すると、
                # リストに含まれる値の出現回数を一度に増やせる
                frequency.update(tokens)
                
                # 10,000件ごとに進捗を表示
                count_proccessed += 1
                if count_proccessed % 10000 == 0:
                    print('{0} documents were proccessed.'.format(count_proccessed),
                         file=sys.stderr)
                    
    # 全記事の処理が完了したら上位30件の名詞と出現回数を表示する
    for token, count in frequency.most_common(30):
        print(token, count)
        
        
def iter_docs(file):
    """
    ファイルオブジェクトを読み込んで、記事の中身(開始タグ<doc ...>と終了タグ</doc>の間のテキスト)を
    順に返すジェネレーター関数
    """
    
    for line in file:
        if line.startswith('<doc '):
            buffer = [] # 開始タグが見つかったらバッファを初期化する
        elif line.startswith('</doc>'):
            # 終了タグが見つかったらバッファの中身を結合してyieldする
            content = ''.join(buffer)
            yield content
        else:
            buffer.append(line) # 開始タグ・終了タグ以外の業はバッファに追加する
            
            
def get_tokens(tagger, content):
    """
    文書内に出現した名詞のリストを取得する関数
    """
    
    tokens = [] # この記事で出現した名詞を格納するリスト
    
    node = tagger.parseToNode(content)
    while node:
        # node.featureはカンマで区切られた文字列なので、split() で分割して
        # 最初の2項目をcategoryとsub_categoryに代入する
        category, sub_category = node.feature.split(',')[:2]
        # 固有名詞または一般名詞の場合のみtokensに追加する
        if category == '名詞' and sub_category in ('固有名詞', '一般'):
            tokens.append(node.surface)
        node = node.next
        
    return tokens


if __name__ == '__main__':
    main()

Writing word_frequency.py


In [22]:
!python word_frequency.py articles

Proccessing articles/AA/wiki_00...
10000 documents were proccessed.
Proccessing articles/AA/wiki_01...
20000 documents were proccessed.
30000 documents were proccessed.
Proccessing articles/AA/wiki_02...
40000 documents were proccessed.
50000 documents were proccessed.
Proccessing articles/AA/wiki_03...
60000 documents were proccessed.
70000 documents were proccessed.
Proccessing articles/AA/wiki_04...
80000 documents were proccessed.
月 298912
日本 130996
時代 65654
駅 51080
世界 46833
作品 44752
番組 43670
列車 43488
昭和 42446
東京 41921
一般 38469
地域 37079
鉄道 37058
平成 36889
中心 36282
アメリカ 36205
ホーム 31774
世紀 30718
バス 30405
大学 30271
車両 29862
間 29861
路線 29207
映画 28781
他 28559
学校 27057
形 26538
ドイツ 26472
事業 26380
テレビ 26332


In [23]:
# chapter 5-2 APIによるデータの収集と活用

In [1]:
# Twitterからのデータの収集

In [2]:
# Requests-OAuthlibを使ったTwitter REST APIの利用

In [3]:
# 認証情報は環境変数に記述して、そこから取得するようにする

In [8]:
%%writefile rest_api_with_requests_oauthlib.py

# Requests-OAuthlibを使ってタイムラインを取得する

import os
from requests_oauthlib import OAuth1Session

# 環境変数から認証情報を取得する
CONSUMER_KEY = os.environ['CONSUMER_KEY']
CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

# 認証情報を使ってOAuth1Sessionオブジェクトを得る
twitter = OAuth1Session(CONSUMER_KEY,
                       client_secret=CONSUMER_SECRET,
                       resource_owner_key=ACCESS_TOKEN,
                       resource_owner_secret=ACCESS_TOKEN_SECRET)

# ユーザーのタイムラインを取得する
response = twitter.get('https://api.twitter.com/1.1/statuses/home_timeline.json')

# APIのレスポンスはJSON形式なので、response.json()でパースしてlistを取得できる
# statusはツイート(Twitter APIではStatusと呼ばれる)を表すdict
for status in response.json():
    print('@' + status['user']['screen_name'], status['text']) # ユーザー名とツイートを表示する

Overwriting rest_api_with_requests_oauthlib.py


In [ ]:
!forego run python rest_api_with_requests_oauthlib.py

In [10]:
# TweepyによるTwitter REST APIの利用

In [13]:
%%writefile rest_api_withtweepy.py

# Tweepyを使ってタイムラインを取得する

import os
import tweepy

# 環境変数から認証情報を取得する
CONSUMER_KEY = os.environ['CONSUMER_KEY']
CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

# 認証情報を設定する
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth) # APIクライアントを取得する
public_tweets = api.home_timeline() # ユーザーのタイムラインを取得する

# 得られるツイートのオブジェクトはTweepyのStatusオブジェクト
for status in public_tweets:
    print('@' + status.user.screen_name, status.text) # ユーザー名とツイートを取得する

Overwriting rest_api_withtweepy.py


In [ ]:
!forego run python rest_api_withtweepy.py

In [15]:
# TweepyによるTwitter Streaming APIの利用

In [21]:
%%writefile streaming_api_with_tweepy.py

# TweepyによるStreaming APIの利用

import os
import tweepy

# 環境変数から認証情報を取得する
CONSUMER_KEY = os.environ['CONSUMER_KEY']
CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

# 認証情報を設定する
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)


class MyStreamListener(tweepy.StreamListener):
    """
    Streaming APIで取得したツイートを処理するためのクラス
    """
    
    def on_status(self, status):
        """
        ツイートを受信した時に呼び出されるメソッド
        引数はツイートを表すStatusオブジェクト
        """
        print('@' + status.author.screen_name, status.text)
        
        
# 認証情報とStreamListenerを指定してStreamオブジェクトを取得する
stream = tweepy.Stream(auth, MyStreamListener())
# 公開されているツイートをサンプリングしたストリームを受信する
# キーワード引数languagesで日本語のツイートのみに絞り込む
stream.sample(languages=['ja'])

Overwriting streaming_api_with_tweepy.py


In [19]:
# !forego run python streaming_api_with_tweepy.py
# 次々にツイートが表示されるので、実行はターミナルで
# キャンセルは ctrl + C

In [22]:
# Amazonの商品情報の収集

In [23]:
# Amazonアソシエイト・プログラムに登録するのが面倒なので後回し

In [24]:
# YouTubeからの動画情報の収集

In [25]:
# ターミナル上のcurlコマンドでYouTube Data APIを使う
# 面倒なので後回し

In [26]:
# Google API Client for Pythonを使う

In [32]:
%%writefile search_youtube_videos.py

# YouTubuの動画を検索する

import os
from apiclient.discovery import build

YOUTUBE_API_KEY = os.environ['YOUTUBE_API_KEY'] # 環境変数からAPIキーを取得する

# YouTubeのAPIクライアントを組み立てる
# build()関数の第1引数にはAPI名を、第2引数にはAPIのバージョンを指定し、
# キーワード引数developerKeyでAPIキーを指定する
# この関数には内部的にYouTube API用のURLにアクセスし、APIのリソースやメソッドの情報を取得する
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

# キーワード引数で引数を指定し、search.listメソッドを呼び出す
# list()メソッドでgoogleapiclient.http.HttpRequestオブジェクトが得られ、
# execute()メソッドを実行すると実際にHTTPリクエストが送られて、APIのレスポンスが得られる
search_response = youtube.search().list(
    part='snippet',
    q='手芸',
    type='video',
).execute()

# search_responseはAPIのレスポンスのJSONをパースしたdict
for item in search_response['items']:
    print(item['snippet']['title']) # 動画のタイトルを表示する

Overwriting search_youtube_videos.py


In [33]:
!forego run python search_youtube_videos.py

【購入品紹介】お気に入りの手芸屋さんで買い物♡大阪の可愛いお店
#プチプラDIY ダイソーのスエード調手芸ひもが可愛すぎる！
簡単！カチュームの作り方／広島 あとりえChikuTaku(ちくたくの手芸塾)
【手芸】トイレットペーパーの芯でミニチュアのニット帽を作る。
針も糸も使わない手芸！果たしてそんなことが可能なのか？作品のクオリティと簡単さにドランクドラゴンも驚愕！！【公式】


In [34]:
# 動画情報をMongoDBに格納して検索する